In [1]:
from GraphWithAsynchronousDynamics import *

In [2]:
class SISAsynchronousDynamicsRewire(GraphWithAsynchronousDynamics):
    '''A graph with a particular SIS dynamics. We use probabilities
    to express infection and recovery per timestep, and run the system
    using Asynchronous dynamics.'''
    
    # the possible dynamics states of a node for SIR dynamics
    SUSCEPTIBLE = 'susceptible'
    INFECTED = 'infected'
    
    # list of infected nodes, the sites of all the dynamics
    _infected = []
    _susceptible = []
    _recovered = []
    
    
    def __init__( self, time_limit = 10000, p_infect = 0.0, p_recover = 1.0, p_infected = 0.0, p_rewire = 0.0, graph = None ):
        '''Generate a graph with dynamics for the given parameters.
        
        pInfect: infection probability (defaults to 0.0)
        pRecover: probability of recovery (defaults to 1.0)
        pInfected: initial infection probability (defaults to 0.0)
        g: the graph to copy from (optional)'''
        states = {self.SUSCEPTIBLE,self.INFECTED}
        rates = dict()
        rates['p_infect'] = p_infect
        rates['p_recover'] = p_recover
        rates['p_infected'] = p_infected
        rates['p_rewire'] = p_rewire
        GraphWithAsynchronousDynamics.__init__(self, time_limit = time_limit, graph = graph, states = states, rates = rates)
        self._p_infect = p_infect
        self._p_recover = p_recover
        self._p_infected = p_infected
        self._p_rewire = p_rewire
            
    def _before( self ):
        '''Seed the network with infected nodes, and mark all edges
        as unoccupied by the dynamics.'''
        self._infected = []       # in case we re-run from a dirty intermediate state
        for n in self.node.keys():
            if numpy.random.random() <= self._p_infected:
                self.node[n][self.DYNAMICAL_STATE] = self.INFECTED
            else:
                self.node[n][self.DYNAMICAL_STATE] = self.SUSCEPTIBLE
        for (n, m, data) in self.edges_iter(data = True):
            data[self.OCCUPIED] = False
    
    def _after(self):
        pass
    
    def set_timestep_rate(self):
        '''Calculate the minimum time-jump. Choose the maximum from the recovery rate
        or the max node degree * the infection rate. Timestep will be 1 over this value'''
        max_degree = max(self.degree().values())
        return max(max_degree*self._p_infect, max_degree*self._p_rewire, self._p_recover)
     
    def at_equilibrium( self ):
        '''SIR dynamics is at equilibrium if there are no more
        infected nodes left in the network or if we've exceeded
        the default simulation length.
        
        returns: True if the model has stopped'''
        
        '''if(t%1000 == 0.0):
            print("Processing timestep ",t)'''
        
        if self.CURRENT_TIMESTEP >= self._time_limit:
            return True
        else:
            return (len(self.POPULATION[self.INFECTED]) == 0)
            
    
    def asyn_node_action(self, node = 0, dt = 0.0, r = 0.0):
        
        # If chosen node is infected
        if(self.node[node][self.DYNAMICAL_STATE] == self.INFECTED):
            # If random number lower than Number of nodes x rate recovery * timestep
            if(r < self.order()*self._p_recover*dt):
                # Node recovers
                self.update_node(node,self.INFECTED,self.SUSCEPTIBLE)
                return True
        # If chosen node is susceptible
        elif(self.node[node][self.DYNAMICAL_STATE] == self.SUSCEPTIBLE):
            neighbours = self.neighbors(node)
            infected_neighbours = [ ne for ne in neighbours if self.node[ne][self.DYNAMICAL_STATE] == self.INFECTED ]
            num_inf_neighb = len(infected_neighbours)
            
            infect_chance = self._p_infect*num_inf_neighb
            rewire_chance = self._p_rewire*num_inf_neighb
            if(r < self.order()*dt*infect_chance):
                # Node gets infected
                self.update_node(node,self.SUSCEPTIBLE,self.INFECTED)
                return True
            elif(r < self.order()*dt*infect_chance + self.order()*dt*rewire_chance):
                # Node rewires
                self.rewire(node, neighbours, infected_neighbours)
        
        return False
    
    def rewire(self, n = 0 , neighbours = [], infected_neighbours = []):
        
        # Remove a link to an infected neighbour
        neighb_index = numpy.random.randint(len(infected_neighbours))
        self.remove_edges_from([(n, infected_neighbours[neighb_index])])
        
        # Add a link to a new node
        
        # Clone the list of susceptible nodes - these will be the candidates for the new link
        candidates = list(self.POPULATION[self.SUSCEPTIBLE])
                        
        # Remove the current node (don't want self loops)
        if n in candidates:
            candidates.remove(n)
        
         # Remove neighbours from candidates        
        candidates = [c for c in candidates if c not in neighbours]
       
        # Pick a candidate
        if(len(candidates) >= 1):
            new_neighb_index = numpy.random.randint(len(candidates))
            self.add_edge(n, candidates[new_neighb_index])
        
        # Network topology has changed so alter the timestep rate
        max_trans_rate = self.set_timestep_rate()
        
        # Timestep = 1 over number of nodes by maximum transition rate
        self.DT = 1.0/(self.order()*max_trans_rate);
            